# Getting NASA Meteorological information:

Este notebook explora la posibilidad de descargar la información de la NASA. Posteriormente se debe "productivizar" por lo que servirá como base para el futuro. 

La descripción de la informacion meteorológica se obtuvo de este [link](https://gist.github.com/abelcallejo/d68e70f43ffa1c8c9f6b5e93010704b8).

# Imports

In [1]:
import datetime 
import json
import pandas as pd
import requests

from src.data.utils import get_general_path, join_paths, check_if_filepath_exists, save_as_pickle


## Configs

## Constants

In [2]:
INTERIM_PATH = 'data/interim'
RAW_PATH = 'data/raw'
SIMPLIFIED_MUNICIPAL_DATA = 'municipal_data.pkl'
METEOROLOGICAL_PATH = 'meteorological_info'

INITIAL_DATE = '2000/01/01'
FINAL_DATE = '2024/06/15'

In [3]:
PARAMETERS_DEFINITION = {
    'PS': 'The average of surface pressure at the surface of the earth.',
    'RH2M': 'Relative Humidity at 2 Meters: The ratio of actual partial pressure of water vapor to the partial pressure at saturation, expressed in percent.',
    'RHOA': 'The air density at surface of the earth.', 
    'T10M': 'The air (dry bulb) temperature at 10 meters above the surface of the earth.',
    'T10M_MAX': 'The maximum hourly air (dry bulb) temperature at 10 meters above the surface of the earth in the period of interest.',
    'T10M_MIN': 'The minimum hourly air (dry bulb) temperature at 10 meters above the surface of the earth in the period of interest.',
    'T2M': 'The average air (dry bulb) temperature at 2 meters above the surface of the earth.',
    'T2M_MAX':'The maximum hourly air (dry bulb) temperature at 2 meters above the surface of the earth in the period of interest.',
    'T2M_MIN':'The minimum hourly air (dry bulb) temperature at 2 meters above the surface of the earth in the period of interest.',
    'PRECTOTCORR': 'The bias corrected average of total precipitation at the surface of the earth in water mass (includes water content in snow).',
    'TQV': 'The total atmospheric water vapor contained in a vertical column of unit cross-sectional area extending from the surface to the top of the atmosphere.',
    'T2MWET': 'The adiabatic saturation temperature which can be measured by a thermometer covered in a water-soaked cloth over which air is passed at 2 meters above the surface of the earth.',
    'T2MDEW': 'The dew/frost point temperature at 2 meters above the surface of the earth.',
    'TROPQ': 'The ratio of the mass of water vapor to the total mass of air at the tropopause pressure given by TROPPB (kg water/kg total air).',
    'QV2M': 'The ratio of the mass of water vapor to the total mass of air at 2 meters (kg water/kg total air).',
    'TO3': "The total amount of ozone in a column extending vertically from the earth's surface to the top of the atmosphere.",
    'TROPT': 'The air (dry bulb) temperature at the tropopause pressure given by TROPPB.',
    'TROPPB': 'The tropopause pressure based on the blended estimate.',
    'TS': "The average temperature at the earth's surface.",
    
}


## Functions

In [4]:
def get_url(lon, lat, start, end):
    parameter_list = PARAMETERS_DEFINITION.keys()
    base_url = "https://power.larc.nasa.gov/api/temporal/daily/point?"
    params = f"parameters={(',').join(parameter_list)}"
    community = '&community=RE'
    longitude = f"&longitude={lon}"
    latitude = f"&latitude={lat}"
    start_date = f"&start={start.strftime('%Y%m%d')}"
    end_date = f"&end={end.strftime('%Y%m%d')}"
    final_format= '&format=JSON'
    url = f'{base_url}{params}{community}{longitude}{latitude}{start_date}{end_date}{final_format}'
    return url

def find_url_with_point(point, start_date, end_date):
    lon, lat = point.x, point.y
    url = get_url(lon=lon, lat=lat, start=start_date, end=end_date)
    return url

def create_meteorological_filename(mun_id):
    general_path = get_general_path()
    filename = f'{mun_id}.pkl'
    path = join_paths(general_path, RAW_PATH, METEOROLOGICAL_PATH, filename)
    return path

def check_if_meteorological_info_downloaded(mun_id):
    path = create_meteorological_filename(mun_id)
    exists = check_if_filepath_exists(path)
    if not exists:
        print(f'Meteorological information for {mun_id} does not exists... creating one at: {path}')
        return path
    print(f'Meteorological information for {mun_id} already saved at: {path}')

def get_meteorological_info(url):
    text_information = requests.get(url).text
    formated_text_information = json.loads(text_information)
    return formated_text_information

def save_meteorological_information(information, path): 
    save_as_pickle(what=information, where=path)

    
def extract_and_save_meteorological_information(row_iterated):
    mun_id, row_info = row_iterated
    link = row_info.information_link
    path = check_if_meteorological_info_downloaded(mun_id)
    if path is None:
        print(f'Done with {mun_id}.')
    else:
        information = get_meteorological_info(link)
        save_meteorological_information(information, path)
        print(f'Saved is accomplished: Done with {mun_id}.')
    return None

## Read data

In [5]:
general_path = get_general_path()
municipal_data_path = join_paths(general_path, INTERIM_PATH, SIMPLIFIED_MUNICIPAL_DATA)
municipal_data = pd.read_pickle(municipal_data_path)


## Process Data

In [6]:
start_date = pd.to_datetime('2000/01/01') ## Three years before the first day in droughts data
#start_date = pd.to_datetime('2024/06/13')
end_date = pd.to_datetime('2024/06/15')
#end_date = datetime.datetime.now() + datetime.timedelta(days=-4)
#print(get_url(lon=-103.38479, lat=20.72356, start=start, end=end))

In [7]:
municipal_data['information_link'] = municipal_data.representative_coordinate.apply(
    find_url_with_point, start_date=start_date, end_date=end_date
)

In [13]:
for row in municipal_data.iterrows():
    extract_and_save_meteorological_information(row)

Meteorological information for 01_008 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/01_008.pkl
Done with 01_008.
Meteorological information for 01_009 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/01_009.pkl
Done with 01_009.
Meteorological information for 01_010 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/01_010.pkl
Done with 01_010.
Meteorological information for 01_011 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/01_011.pkl
Done with 01_011.
Meteorological information for 01_001 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/01_001.pkl
Done with 01_001.
Meteorological information for 01_002 already saved at: /mnt/c/Users/dhdzm/Documents/projects/s

Meteorological information for 13_064 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/13_064.pkl
Done with 13_064.
Meteorological information for 13_051 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/13_051.pkl
Done with 13_051.
Meteorological information for 13_052 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/13_052.pkl
Done with 13_052.
Meteorological information for 13_053 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/13_053.pkl
Done with 13_053.
Meteorological information for 13_054 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/13_054.pkl
Done with 13_054.
Meteorological information for 13_055 already saved at: /mnt/c/Users/dhdzm/Documents/projects/s

Meteorological information for 20_015 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_015.pkl
Done with 20_015.
Meteorological information for 20_020 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_020.pkl
Done with 20_020.
Meteorological information for 20_566 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_566.pkl
Done with 20_566.
Meteorological information for 20_016 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_016.pkl
Done with 20_016.
Meteorological information for 20_525 already saved at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_525.pkl
Done with 20_525.
Meteorological information for 20_526 already saved at: /mnt/c/Users/dhdzm/Documents/projects/s

Saved is accomplished: Done with 20_283.
Meteorological information for 20_286 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_286.pkl
Saved is accomplished: Done with 20_286.
Meteorological information for 20_295 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_295.pkl
Saved is accomplished: Done with 20_295.
Meteorological information for 20_285 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_285.pkl
Saved is accomplished: Done with 20_285.
Meteorological information for 20_288 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_288.pkl
Saved is accomplished: Done with 20_288.
Meteorological information for 20_289 does not exists... creating one a

Saved is accomplished: Done with 20_327.
Meteorological information for 20_328 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_328.pkl
Saved is accomplished: Done with 20_328.
Meteorological information for 20_335 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_335.pkl
Saved is accomplished: Done with 20_335.
Meteorological information for 20_336 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_336.pkl
Saved is accomplished: Done with 20_336.
Meteorological information for 20_339 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/20_339.pkl
Saved is accomplished: Done with 20_339.
Meteorological information for 20_341 does not exists... creating one a

...

Saved is accomplished: Done with 30_191.
Meteorological information for 30_192 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/30_192.pkl
Saved is accomplished: Done with 30_192.
Meteorological information for 30_194 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/30_194.pkl
Saved is accomplished: Done with 30_194.
Meteorological information for 30_199 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/30_199.pkl
Saved is accomplished: Done with 30_199.
Meteorological information for 30_200 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/30_200.pkl
Saved is accomplished: Done with 30_200.
Meteorological information for 30_201 does not exists... creating one a

Saved is accomplished: Done with 31_065.
Meteorological information for 31_067 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_067.pkl
Saved is accomplished: Done with 31_067.
Meteorological information for 31_101 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_101.pkl
Saved is accomplished: Done with 31_101.
Meteorological information for 31_102 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_102.pkl
Saved is accomplished: Done with 31_102.
Meteorological information for 31_069 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_069.pkl
Saved is accomplished: Done with 31_069.
Meteorological information for 31_071 does not exists... creating one a

Saved is accomplished: Done with 31_049.
Meteorological information for 31_050 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_050.pkl
Saved is accomplished: Done with 31_050.
Meteorological information for 31_025 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_025.pkl
Saved is accomplished: Done with 31_025.
Meteorological information for 31_026 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_026.pkl
Saved is accomplished: Done with 31_026.
Meteorological information for 31_027 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_027.pkl
Saved is accomplished: Done with 31_027.
Meteorological information for 31_031 does not exists... creating one a

Saved is accomplished: Done with 31_087.
Meteorological information for 31_088 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_088.pkl
Saved is accomplished: Done with 31_088.
Meteorological information for 31_091 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_091.pkl
Saved is accomplished: Done with 31_091.
Meteorological information for 31_094 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_094.pkl
Saved is accomplished: Done with 31_094.
Meteorological information for 31_095 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/31_095.pkl
Saved is accomplished: Done with 31_095.
Meteorological information for 31_096 does not exists... creating one a

Saved is accomplished: Done with 32_054.
Meteorological information for 32_024 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/32_024.pkl
Saved is accomplished: Done with 32_024.
Meteorological information for 32_031 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/32_031.pkl
Saved is accomplished: Done with 32_031.
Meteorological information for 32_037 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/32_037.pkl
Saved is accomplished: Done with 32_037.
Meteorological information for 32_040 does not exists... creating one at: /mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/raw/meteorological_info/32_040.pkl
Saved is accomplished: Done with 32_040.
Meteorological information for 32_018 does not exists... creating one a